# Prompt Templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM.

In [20]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")
model = ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)

In [ ]:
# prompt with more complexity
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         "You are a helpful assistant. Answer all the question to the best of your abilities in {language}"
        ),
        MessagesPlaceholder(variable_name="messages") #It acts as a slot in your prompt template where a list of messages (like from a memory object or manual input) can be inserted when you call the chain.

    ]
)
chain = prompt|model

In [22]:
from langchain_core.messages import HumanMessage
chain.invoke({"messages":[HumanMessage(content="Hi my name is Aneesh")],"language":"Hindi"})

AIMessage(content='Namaste Aneesh! \n\nआपका स्वागत है! मैं आपकी मदद करने के लिए यहाँ हूँ।  आप मुझसे जो भी प्रश्न पूछना चाहें, मैं अपने ज्ञान के अनुसार उत्तर देने की पूरी कोशिश करूँगा।  \n\nक्या आप कोई प्रश्न पूछना चाहेंगे? \n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 74, 'prompt_tokens': 33, 'total_tokens': 107, 'completion_time': 0.134545455, 'prompt_time': 0.002365704, 'queue_time': 0.232555655, 'total_time': 0.136911159}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-3e4bbb51-8392-49d6-9e25-8276f799107c-0', usage_metadata={'input_tokens': 33, 'output_tokens': 74, 'total_tokens': 107})

In [23]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory

store = {}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]



Let's now wrap this more complicated chain in a Message History Class. This time, because there are multiple keys in the input, we need to specify the correct key to save chat history.

In [26]:
from langchain_core.runnables.history import RunnableWithMessageHistory
with_message_history = RunnableWithMessageHistory(chain,
                                                  get_session_history,
                                                  input_messages_key="messages"
                                                  )

In [ ]:
config = {"configurable":{"session_id":"chat3"}}
response = with_message_history.invoke(
    {"messages":[HumanMessage(content="Hi my name is Aneesh")],"language":"Hindi"},
    config=config
)
response.content

'नमस्ते Aneesh! 👋  मुझे आपकी मदद करने में खुशी हो रही है। \n\nआप क्या जानना चाहेंगे? 😊  कृपया कोई भी प्रश्न पूछने में संकोच न करें।  \n\n'

In [30]:
with_message_history.invoke(

    {"messages":[HumanMessage(content="what's my name?")],"language":"French"},
    config=config
)


AIMessage(content='Votre nom est Aneesh.  😊 \n\nIs there anything else I can help you with? \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 150, 'total_tokens': 174, 'completion_time': 0.043636364, 'prompt_time': 0.006016721, 'queue_time': 0.328995669, 'total_time': 0.049653085}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-67ae6a18-9fbc-459c-9253-70c61d9140b8-0', usage_metadata={'input_tokens': 150, 'output_tokens': 24, 'total_tokens': 174})